<a href="https://colab.research.google.com/github/C241-PS517-ISALAT/Isalat-Model/blob/model_debug/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cv2

from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
train_data = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/My Drive/bisindo/data/train')
validation_data = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/My Drive/bisindo/data/validation')

Found 234 files belonging to 26 classes.
Found 78 files belonging to 26 classes.


In [6]:
for images, labels in train_data.take(1):
  print(images.shape)  # Should print: (batch_size, 300, 300, 3)

(32, 300, 300, 3)


In [17]:
#build a model

model = Sequential()
model.add(Conv2D(16,input_shape=(300,300,3),kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(GlobalAveragePooling2D())
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 300, 300, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 150, 150, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 150, 150, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 75, 75, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 37, 37, 64)       

In [28]:
#model train
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=tf.keras.optimizers.RMSprop(lr=0.001))
model.fit(train_data,epochs=25)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


8/8 [==============================] - 5s 69ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 2/25
8/8 [==============================] - 4s 61ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 3/25
8/8 [==============================] - 4s 64ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 4/25
8/8 [==============================] - 4s 68ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 5/25
8/8 [==============================] - 5s 58ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 6/25
8/8 [==============================] - 4s 59ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 7/25
8/8 [==============================] - 4s 61ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 8/25
8/8 [==============================] - 4s 64ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 9/25
8/8 [==============================] - 4s 60ms/step - loss: 0.0000e+00 - accuracy: 0.0385
Epoch 10/25
8/8 [==============================] - 4s 64ms/step - loss: 0.0000e+00 - accuracy: 0.0385


In [29]:
gap_weights = model.layers[-1].get_weights()[0]
gap_weights.shape

cam_model  = Model(inputs=model.input,outputs=(model.layers[-3].output,model.layers[-1].output))
cam_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8_input (InputLayer  [(None, 300, 300, 3)]     0         
 )                                                               
                                                                 
 conv2d_8 (Conv2D)           (None, 300, 300, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 150, 150, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 150, 150, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 75, 75, 32)        0         
 g2D)                                                            
                                                             

In [31]:
def show_cam(image_value, features, results):
  '''
  Displays the class activation map of an image

  Args:
    image_value (tensor) -- preprocessed input image with size 300 x 300
    features (array) -- features of the image, shape (1, 37, 37, 128)
    results (array) -- output of the sigmoid layer
  '''

  # there is only one image in the batch so we index at `0`
  features_for_img = features[0]
  prediction = results[0]

  # there is only one unit in the output so we get the weights connected to it
  class_activation_weights = gap_weights[:,0]

  # upsample to the image size
  class_activation_features = sp.ndimage.zoom(features_for_img, (300/37, 300/37, 1), order=2)

  # compute the intensity of each feature in the CAM
  cam_output  = np.dot(class_activation_features,class_activation_weights)

  # visualize the results
  print(f'sigmoid output: {results}')
  print(f"prediction: {'dog' if round(results[0][0]) else 'cat'}")
  plt.figure(figsize=(8,8))
  plt.imshow(cam_output, cmap='jet', alpha=0.5)
  plt.imshow(tf.squeeze(image_value), alpha=0.5)
  plt.show()


In [32]:
# utility function to preprocess an image and show the CAM
def convert_and_classify(image):

  # load the image
  img = cv2.imread(image)

  # preprocess the image before feeding it to the model
  img = cv2.resize(img, (300,300)) / 255.0

  # add a batch dimension because the model expects it
  tensor_image = np.expand_dims(img, axis=0)

  # get the features and prediction
  features,results = cam_model.predict(tensor_image)

  # generate the CAM
  show_cam(tensor_image, features, results)

In [36]:
for image in validation_data.take(1):
  convert_and_classify(image)

TypeError: Can't convert object to 'str' for 'filename'